In [4]:
import os
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd 

## Load persisted model and dataset

In [3]:
# load model
model_dir = "./models"
model_h5_file = os.path.join(model_dir, "model_task_2_cnn_classification.h5")

model = load_model(model_h5_file)
model

In [14]:
# Load Dataset
data_file_path = "./data/similarities.parquet"
df = pd.read_parquet(data_file_path)

# Note: Numpy array is treated as string here 
df.dtypes

path                object
furniture           object
style               object
histogram           object
texture_feature     object
compactness        float64
dtype: object

## TODO: Prediction